In [ ]:
!pip install flask-ngrok
!pip install flask pyngrok

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Load and preprocess dataset
portfolio_data = pd.read_csv('/content/investment_dataset.csv')

# Remove non-numeric columns or encode categorical variables
# Example: If 'Risk' is categorical, encode it using one-hot encoding
portfolio_data = pd.get_dummies(portfolio_data, columns=['Risk'])

# Select available numeric columns for RL model input
# Use the actual column names present in your dataset
numeric_features = ['Age', 'Salary_lpa', 'SIP']  # Adjust based on your actual columns
portfolio_data_numeric = portfolio_data[numeric_features]

# Define RL environment parameters
state_size = len(numeric_features)
action_size = 5
learning_rate = 0.001

# Define RL model architecture
def build_model(state_size, action_size, learning_rate):
    model = Sequential([
        Dense(32, input_dim=state_size, activation='relu'),
        Dense(32, activation='relu'),
        Dense(action_size, activation='linear')
    ])
    model.compile(loss='mse', optimizer=Adam(learning_rate))
    return model

# Initialize RL model
rl_model = build_model(state_size, action_size, learning_rate)

# Training loop
num_episodes = 1000
for episode in range(num_episodes):
    state = portfolio_data_numeric.sample(1)
    state = state.values.astype(np.float32)
    action = rl_model.predict(state)
    # Reward calculation and next state simulation logic goes here
    # rl_model.fit(state, action, epochs=1, verbose=0)  # Uncomment for actual training

# Example functions to implement:
# - calculate_reward(state, action)
# - simulate_next_state(state, action)

# Save trained model
rl_model.save('rl_portfolio_diversification_model.h5')


In [ ]:
!pip install flask pyngrok yfinance matplotlib

from flask import Flask, request, render_template_string
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from pyngrok import ngrok
import yfinance as yf
import matplotlib.pyplot as plt

# Initialize Flask application
app = Flask(__name__)

# Load the trained RL model
rl_model = load_model('rl_portfolio_diversification_model.h5')

# HTML template for portfolio recommendation and stock information
html_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Portfolio Dashboard</title>
    <style>
         body {
            background-color: #282c34; /* Updated background color */
            color: white;
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
        }
        .container {
            display: flex;
            justify-content: space-between; /* Spread columns */
            max-width: 1200px; /* Increased max-width for larger container */
            width: 100%;
            border: 2px solid #ccc; /* Border to separate columns */
            padding: 20px;
            background-color: rgba(255, 255, 255, 0.1); /* Semi-transparent white background */
            border-radius: 10px;
            box-shadow: 0 0 20px rgba(0, 0, 0, 0.2); /* Shadow effect */
        }
        .column {
            width: 48%; /* Adjust width to leave space for the vertical divider */
            padding: 20px;
            background-color: rgba(0, 123, 255, 0.3); /* Blue tint background */
            border-radius: 10px;
        }
        .column:first-child {
            border-right: 1px solid #ccc; /* Vertical divider for first column */
        }
        .column h2 {
            font-size: 24px; /* Increased font size for headings */
            font-style: italic; /* Italic font style */
            margin-bottom: 15px;
        }
        form {
            margin-top: 20px;
        }
        /* Modal and other styles as needed */
        .modal {
            display: none; /* Hidden by default */
            position: fixed; /* Stay in place */
            z-index: 1; /* Sit on top */
            left: 0;
            top: 0;
            width: 100%; /* Full width */
            height: 100%; /* Full height */
            overflow: auto; /* Enable scroll if needed */
            background-color: rgba(0,0,0,0.4); /* Black w/ opacity */
        }
        .modal-content {
            background-color: #fefefe;
            margin: 15% auto; /* 15% from the top and centered */
            padding: 20px;
            border: 1px solid #888;
            width: 80%; /* Could be more or less, depending on screen size */
            border-radius: 10px;
        }
        .close {
            color: #aaa;
            float: right;
            font-size: 28px;
            font-weight: bold;
            cursor: pointer;
        }
        .close:hover,
        .close:focus {
            color: black;
            text-decoration: none;
        }

    </style>
</head>
<body>
    <h1>Portfolio Dashboard</h1>
    <div class="container">
        <div class="column">
            <h2>Portfolio Diversification Recommendation</h2>
            <form action="/recommend" method="post">
                <label for="age">Age:</label>
                <input type="number" id="age" name="age" required><br><br>
                <label for="salary">Salary (in LPA):</label>
                <input type="number" id="salary" name="salary_lpa" required><br><br>
                <label for="sip">SIP (monthly in INR):</label>
                <input type="number" id="sip" name="sip" required><br><br>
                <label for="risk">Risk:</label>
                <select id="risk" name="risk" required>
                    <option value="Low">Low</option>
                    <option value="Medium">Medium</option>
                    <option value="High">High</option>
                </select><br><br>
                <label for="years">Investment Period (years):</label>
                <input type="number" id="years" name="years" required><br><br>
                <label for="expected_return">Expected Returns (%):</label>
                <input type="number" id="expected_return" name="expected_return" required><br><br>
                <button type="submit">Get Recommendation</button>
            </form>
            {% if result %}
            <h3>Recommended Allocation</h3>
            <p>Equity: {{ result.Equity }}%</p>
            <p>Bonds: {{ result.Bonds }}%</p>
            <p>FD: {{ result.FD }}%</p>
            <p>Real Estate: {{ result.RealEstate }}%</p>
            <p>Gold: {{ result.Gold }}%</p>
            <h3>Investment Projection</h3>
            <p>Total Corpus after {{ years }} years at {{ expected_return }}% returns: INR {{ corpus }}</p>
            {% endif %}
        </div>
        <div class="column">
            <h2>Stock Information</h2>
            <form action="/stock_info" method="get">
                <label for="stock_name">Enter Stock Name (e.g., AAPL):</label>
                <input type="text" id="stock_name" name="stock_name" required><br><br>
                <button type="submit">Get Stock Info</button>
            </form>
            {% if stock_info %}
            <h3>Real-time Stock Information</h3>
            <p>Current Price: {{ stock_info['Current Price'] }}</p>
            <p>Change: {{ stock_info['Change'] }}</p>
            <p>Change Percent: {{ stock_info['Change Percent'] }}%</p>
            <h3>Historical Stock Chart</h3>
            <img src="{{ url_for('static', filename='plot.png') }}" alt="Stock Chart" style="max-width: 100%;">
            {% endif %}
        </div>
    </div>
</body>
</html>
'''

# Route to render the form
@app.route('/')
def index():
    return render_template_string(html_template)

# Route to handle form submission and give recommendations
@app.route('/recommend', methods=['POST'])
def recommend():
    # Get form data
    age = int(request.form['age'])
    salary_lpa = float(request.form['salary_lpa'])
    sip = float(request.form['sip'])
    risk = request.form['risk']
    years = int(request.form['years'])
    expected_return = float(request.form['expected_return'])

    # Prepare input for prediction (assuming you preprocess similarly as during training)
    input_data = pd.DataFrame({
        'Age': [age],
        'Salary_lpa': [salary_lpa],
        'SIP': [sip]
    })

    # Predict action (diversification recommendation) using the loaded model
    state = input_data.values.astype(np.float32)
    action = rl_model.predict(state)[0]  # Get the first (and only) row of prediction

    # Ensure all values are non-negative
    action = np.maximum(action, 0)

    # Normalize action percentages to ensure they sum up to 100%
    total = sum(action)
    if total > 0:  # Avoid division by zero
        normalized_action = [round(a / total * 100, 2) for a in action]
    else:
        normalized_action = [0] * len(action)

    # Adjust normalized values to ensure they sum up to exactly 100%
    total_normalized = sum(normalized_action)
    if total_normalized != 100:
        diff = 100 - total_normalized
        max_index = np.argmax(normalized_action)
        normalized_action[max_index] += diff

    # Calculate the corpus using SIP formula with compounding interest
    monthly_rate = expected_return / 100 / 12
    months = years * 12
    corpus = sip * (((1 + monthly_rate)**months - 1) / monthly_rate) * (1 + monthly_rate)

    # Prepare result
    result = {
        'Equity': normalized_action[0],
        'Bonds': normalized_action[1],
        'FD': normalized_action[2],
        'RealEstate': normalized_action[3],
        'Gold': normalized_action[4]
    }

    # Render the template with the result
    return render_template_string(html_template, result=result, years=years, expected_return=expected_return, corpus=round(corpus, 2))



@app.route('/stock_info')
def stock_info():
    stock_name = request.args.get('stock_name')

    # Get real-time stock information
    stock_info = get_stock_info(stock_name)

    # Plot historical stock chart
    if stock_info:
        plot_stock_chart(stock_name)

    # Render the template with stock information
    return render_template_string(html_template, stock_info=stock_info)

def get_stock_info(stock_name):
    try:
        # Download stock information using yfinance
        stock_data = yf.download(stock_name, period="1d", interval="1m")

        # Extract relevant data points (modify as needed)
        current_price = stock_data["Close"][-1]
        change = stock_data["Close"][-1] - stock_data["Open"][-1]
        change_percent = (change / stock_data["Open"][-1]) * 100

        # Create a dictionary with extracted information
        stock_info = {
            "Current Price": current_price,
            "Change": change,
            "Change Percent": change_percent,
        }
        return stock_info
    except Exception as e:
        print(f"Error retrieving data: {e}")
        return None

def plot_stock_chart(stock_name):
    try:
        # Download historical data (adjust period and interval as needed)
        stock_data = yf.download(stock_name, period="max")

        # Plot the closing price over time
        plt.figure(figsize=(10, 6))
        plt.plot(stock_data.index, stock_data["Close"])
        plt.xlabel("Date")
        plt.ylabel("Price")
        plt.title(f"{stock_name} Stock Price Chart")
        plt.savefig('static/plot.png')  # Save plot to a static file
        plt.close()
    except Exception as e:
        print(f"Error plotting chart: {e}")

# Install ngrok authtoken
!ngrok authtoken 2ihSOqCIhkaZ6Wpuj7SylweBxSb_4oQMtaKpD5V3d7B6gMv6a

# Start ngrok tunnel
url = ngrok.connect(5000)
print(f" * Running on {url}")

# Run the Flask application
app.run()